In [12]:
import pandas as pd
import re
from datetime import datetime, timedelta
from docx import Document
from docx.shared import Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.shared import Inches
from docx.oxml.ns import qn
from docx.oxml import OxmlElement

# Define a function to extract dates in 'dd/mm/yyyy' format within a specified year range
def extract_dates(series, start_year=2014, end_year=2024):
    dates = []
    for value in series:
        if isinstance(value, str):  # Check if the value is a string
            # Find all matches of the date pattern 'dd/mm/yyyy'
            matches = re.findall(r'\b\d{2}/\d{2}/\d{4}\b', value)
            for match in matches:
                year = int(match.split('/')[-1])  # Extract the year from the date
                if start_year <= year <= end_year:
                    dates.append(match)
        elif pd.to_datetime(value, errors='coerce') is not pd.NaT:  # Check if the value is a valid date
            # Convert to 'dd/mm/yyyy' format and add to list
            formatted_date = pd.to_datetime(value).strftime('%d/%m/%Y')
            year = pd.to_datetime(value).year
            if start_year <= year <= end_year:
                dates.append(formatted_date)
    return dates

def extract_email_info(df_schools, school_name):
    # Strip any leading/trailing whitespace from the column names
    df_schools.columns = df_schools.columns.str.strip()

    # Collect all data for that school in particular as a pandas.Series
    school_data = df_schools[df_schools['Organisation Name'] == school_name]

    if school_data.empty:
        raise ValueError(f"School with name '{school_name}' not found.")

    school_data = school_data.iloc[0]

    # Access the pandas.Series to collect all relevant information for the email.

    # Collect School Contact Name
    contact_name = school_data['Contact Name']

    # Collect Contact Email
    contact_email = school_data['Contact Email Adress']

    # Collect the updated contact date
    updated_contact = school_data['Updated contact?']

    # Collect Email General
    email_general = school_data['Email General']

    # Collect School Type
    school_type = school_data['School Type']

    # Collect the name of LGA
    lga_name = school_data['LGA']

    # Extract dates from the school_data Series
    workshop_history = extract_dates(school_data)

    # Convert the list of dates to a pandas Series
    workshop_dates = pd.Series(pd.to_datetime(workshop_history, format='%d/%m/%Y', errors='coerce'))

    # Drop any NaT values
    workshop_dates = workshop_dates.dropna()

    # Collect how many workshops this school has done
    number_of_workshops = len(workshop_dates)

    # Collect the date of the last workshop this school attended
    last_workshop_date = workshop_dates.max()
    
    funding_provider = school_data.get('Funding provider', 'Not Available')
    
    funding_provided = school_data['Funding']

    # Return all relevant information
    return {
        'School Name':school_name,
        'Contact Name': contact_name,
        'Contact Email': contact_email,
        'Updated Contact Date': updated_contact,
        'Email General': email_general,
        'School Type': school_type,
        'LGA Name': lga_name,
        'Number of Workshops': number_of_workshops,
        'Last Workshop Date': last_workshop_date,
        'Funding Provided':funding_provided,
        'Funding Provider':funding_provider
    }

def add_hyperlink(paragraph, text, url):
    # Create a hyperlink element
    part = paragraph.part
    r_id = part.relate_to(url, 'http://schemas.openxmlformats.org/officeDocument/2006/relationships/hyperlink', is_external=True)

    hyperlink = OxmlElement('w:hyperlink')
    hyperlink.set(qn('r:id'), r_id)

    # Create a run element with the hyperlink text
    run = OxmlElement('w:r')
    rPr = OxmlElement('w:rPr')

    # Style the hyperlink (e.g., make it blue and underlined)
    rStyle = OxmlElement('w:rStyle')
    rStyle.set(qn('w:val'), 'Hyperlink')
    rPr.append(rStyle)

    run.append(rPr)
    run.text = text
    hyperlink.append(run)
    paragraph._p.append(hyperlink)

def create_email(template_info, my_contact_info, my_email, output_file):
    # Generate an email based on the provided information and template.

    # Access all the dataframe information for the template construction.
    contact_name = template_info.get('Contact Name', 'Recipient')
    school_name = template_info.get('School Name', 'N/A')
    last_workshop_date = template_info.get('Last Workshop Date', pd.NaT)
    number_of_workshops = template_info.get('Number of Workshops', 0)
    school_type = template_info.get('School Type', 'N/A')
    funding_provider = template_info.get('Funding Provider', 'N/A')
    funding_provided = template_info.get('Funding Provided', 'no')
    email_general = template_info.get('Email General', '')
    contact_email = template_info.get('Contact Email', 'N/A')
    
    # Convert potential NaT to default value if needed
    if pd.isna(last_workshop_date):
        last_workshop_date = 'N/A'
    else:
        last_workshop_date = last_workshop_date.strftime('%d %B %Y')

    # Define key dates
    key_dates = [datetime(2024, 9, 20), datetime(2024, 1, 29), datetime(2024, 4, 15), datetime(2024, 7, 15), datetime(2024, 10, 7)]
    
    # Get the current date
    current_date = datetime.now()

    # Determine if the current date is within two weeks after any of the key dates
    holiday_break_message = ""
    for key_date in key_dates:
        if key_date <= current_date <= key_date + timedelta(days=14):
            holiday_break_message = "Hope you have had a safe and restful holiday break!"
            break

    # Email salutation
    salutation = f"Dear {contact_name} and {school_name}," if pd.notna(contact_name) else f"Dear {school_name} and To Whom it May Concern,"
    
    # Introduction Engagement Message
    if number_of_workshops > 0:
        intro_engagement_message = f"Our F2D Workshop is back again! The last time you booked was {last_workshop_date}, don't forget the importance of road safety within your community."
    else:
        intro_engagement_message = "You have never booked a Fit To Drive workshop before! Make road safety a priority within your community!"

    # Determine cost
    cost_message = "$300 + GST" if school_type == 'GOV' else "$350 + GST"

    # Determine funding message
    funding_message = ""
    if funding_provided.lower() == 'yes':
        funding_message = f"We have received funding in your local area by {funding_provider} in the past."
        
    # Workshop engagement message
    if number_of_workshops > 1:
        engagement_message = f"As our workshops are in high demand and booking up quickly, we thought we would prioritize contacting your school as you have previously engaged in {number_of_workshops} workshops. Your last workshop was {last_workshop_date}."
    elif number_of_workshops == 1:
        engagement_message = f"As our workshops are in high demand and booking up quickly, we thought we would prioritize contacting your school as you have engaged in one workshop. Your last workshop was {last_workshop_date}."
    else:
        engagement_message = "As our workshops are in high demand and booking up quickly, we thought we would prioritize contacting your school as we would love to give you the opportunity to engage in your first workshop."
  
    # Put it all together to create the formatted email document.
    # Create a new Document
    doc = Document()
    
    # Change all font in the document to be Aptos size 12 font.
    # Assuming font styling is applied correctly

    # Add paragraphs
    p = doc.add_paragraph('SUBJECT LINE: F2D: Make Road safety a priority! Book your road safety workshop today!')
    
    p = doc.add_paragraph(str(email_general))
    if contact_email:
        p = doc.add_paragraph(str(contact_email))
    
    p = doc.add_paragraph(salutation)
    
    # Make the following paragraph bold:
    p = doc.add_paragraph(intro_engagement_message)
    p.runs[0].bold = True
        
    if holiday_break_message:
        doc.add_paragraph(holiday_break_message)
    
    # Introduction
    p = doc.add_paragraph('I am reaching out regarding our F2D Year 11 Workshop.\n\nThe Fit to Drive Foundation is one of Victoria’s leading road safety education providers for young people.')
    
    p = doc.add_paragraph('We pride ourselves on delivering relevant, informative, and interactive workshops with the support of key partners including Victoria Police and Fire Rescue Victoria.')
    
    # Make the following paragraph bold:
    p = doc.add_paragraph('The F2D Year 11 Workshop is Fit to Drive’s half-day incursion for students in Year 11 or equivalent cohorts (including VCAL students or those approximately 16-17 years old).')
    p.runs[0].bold = True
       
    p = doc.add_paragraph('A flyer for the F2D Year 11 Workshop has been attached for your perusal.')
    p = doc.add_picture('F2D_y11_WS_Info_graphic.png', width=Inches(4))
    
    # Create a paragraph
    paragraph = doc.add_paragraph()

    # Add the first part of the paragraph
    paragraph.add_run('The cost to your school is only ')

    # Add the bold part
    bold_run = paragraph.add_run(cost_message)
    bold_run.bold = True

    # Add the remaining part of the paragraph
    paragraph.add_run(', provided we can secure funding from your Local Government.')
    
    p = doc.add_paragraph(funding_message)
    
    p = doc.add_paragraph(engagement_message)
    
    p = doc.add_paragraph('Don’t miss out on this opportunity to bring crucial road safety education to your students.')
    
    # BOOKING INSTRUCTION MESSAGE:
    p = doc.add_heading('TO MAKE A BOOKING:')
    run = p.runs[0]
    run.font.size = Pt(14)
    
    p = doc.add_paragraph('Fill out the booking form on our website:   ')
    add_hyperlink(p, 'Click here', 'https://f2d.oxil.io/workshops/f-2-d-year-11-workshop/4/register')
    
    p = doc.add_paragraph("Alternatively, reply to this email with the following details and I can book the workshop for you:")

    # Adding each item as a bullet point
    bullet_points = [
        "What is your school’s name?",
        "What is your school’s Local Government Area (local council)?",
        "How many students are you looking to book the workshop for?",
        "What are your preferred dates for the workshop? (2-3 dates)",
        "Please include the following school times: start of school, start and end of recess, and start of lunchtime."
    ]

    for point in bullet_points:
        p = doc.add_paragraph(point, style='ListBullet')
    
    p = doc.add_paragraph('If you need more information on the suitability of the program for your students, please don’t hesitate to get in touch.') 

    p = doc.add_paragraph('We’re happy to work with you in delivering the message of road safety to all young road users with their specific needs in mind.')

    p = doc.add_paragraph(f'I look forward to hearing from you. Please contact me on {my_contact_info} if you require any further information.')
    
    # Apply font styling to all paragraphs
    for paragraph in doc.paragraphs:
        for run in paragraph.runs:
            run.font.name = 'Aptos'
            run.font.size = Pt(12)
    
    doc.save(output_file)
    
    return output_file

def filter_schools_by_lga(df, lga_name):
    # Filter the DataFrame by the provided LGA name
    filtered_schools = df[df['LGA'].str.contains(lga_name, case=False, na=False)]
    
    # Return a list of all school names in the filtered DataFrame
    return filtered_schools['Organisation Name'].tolist()

In [60]:
file_path = 'MASTER Workshop Planner 2024.xlsx'

# Specify the sheet name
sheet_name_school_list = 'School List'
sheet_name_lga_funding = '2023 LGA funding'

# Load the Excel sheet into a pandas DataFrame
df_schools = pd.read_excel(file_path, sheet_name=sheet_name_school_list, header=1)
df_lga_funding = pd.read_excel(file_path, sheet_name=sheet_name_lga_funding, header=1)


In [57]:
Melbourne_schools = ['Auburn High School',
 'Camberwell Girls Grammar School',
 'Camberwell Grammar School',
 'Rossbourne School',
 "Ruyton Girls' School",
 'Doncaster Secondary College',
 'Warrandyte High School',
 'Avila College',
 'Berengarra School Pathways Campus',
 'Mount Waverley Secondary College',
 'Box Hill High School',
 'Emmaus College',
 'Forest Hill College',
 'Koonung Secondary College',
 'Mullauna College',
 'Nunawading Christian College Secondary School',
 'Our Lady of Sion College']

school_name = "Killester College"

my_contact_info = '0420 588 629'
my_email = 'charles.morgan@fittodrive.org.vic.au'
file_name = f'Patterson_River_Secondary_F2D_workshop_email.docx'

In [58]:
template_info = extract_email_info(df_schools, school_name)

template_info

{'School Name': 'Killester College',
 'Contact Name': 'Elissa Galante',
 'Contact Email': 'egalante@killester.vic.edu.au',
 'Updated Contact Date': 2023,
 'Email General': 'principal@killester.vic.edu.au',
 'School Type': 'CATH',
 'LGA Name': 'greater dandenong',
 'Number of Workshops': 6,
 'Last Workshop Date': Timestamp('2023-05-09 00:00:00'),
 'Funding Provided': 'Yes',
 'Funding Provider': 'City of Greater Dandenong Council'}

In [16]:
email = create_email(template_info, my_contact_info, my_email, file_name)

In [62]:
df_lga_funding

,LGA,Unnamed: 1,Current Funding Body,Current Contact Person,Row F Last Updated By (insert name & date),Details for current funding situation,Amount,Status,Reporting Requirements,Previous Reporting Dates,...,Unnamed: 13,"T1 & 2, 2021","T3 & 4, 2021",Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22
0,Alpine Shire,Relationship,Bright Rotary Club/Kiewa Mitta Murray Communit...,Patrick O'Shea <wandi86@bigpond.net.au> (Presi...,NaN,Bus transport (combined workshop),NaN,NaN,Email BRC to cover Bright bus transport. Grant...,NaN,...,4,0.0,0.0,4,NaN,NaN,NaN,Check all councils late June,Name,Bendigo Bank Community Grants
1,Ararat,NaN,NaN,NaN,NaN,Funding needed- Roadsafe Central Highlands loo...,NaN,NaN,NaN,NaN,...,3,0.0,0.0,3,NaN,NaN,NaN,NaN,Key Dates,Year round
2,Ballarat,NaN,NaN,Des Hudson <des.hudson@police.vic.gov.au> (Vic...,NaN,Funding needed - (have followed up 30/11 on RC...,NaN,NaN,NaN,NaN,...,14,1.0,1.0,12,NaN,NaN,NaN,NaN,Details,https://cia.communityenterprisefoundation.com....
3,Banyule,Relationship,Banyule City Council,Alison Wood <Alison.Wood@banyule.vic.gov.au> (...,NaN,"Gap funding up to $1,500 annually.",1500,"Case-by-case, seek approval from Council Contact",Email Alison with workshop booking details for...,17 Feb 2022. Visited Macleod Secondary College.,...,19,2.0,0.0,17,NaN,NaN,NaN,NaN,NaN,NaN
4,Bass Coast,NaN,Bass Coast,michael.whelan@basscoast.vic.gov.au,NaN,Formerly no funding - Can we get some ???,NaN,NaN,NaN,NaN,...,5,NaN,NaN,5,NaN,NaN,NaN,NaN,Name,Telstra’s Connected Communities Grant Program
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Wyndham,Relationship,RoadSafe Westgate (via Wyndham City Council),Jim Giddings <jimgiddings51@gmail.com> (Chair ...,NaN,"Gap funding, 3-year agreement yet to be confir...",11700,Current,Don't need to send request for each booking. M...,"6 May 2022. ""The report to Wyndham Council was...",...,21,6.0,1.0,14,NaN,NaN,NaN,NaN,NaN,NaN
76,Yarra,NaN,NaN,NaN,NaN,Small Projects grant round open all year round...,NaN,NaN,NaN,NaN,...,16,NaN,NaN,16,NaN,NaN,NaN,NaN,NaN,NaN
77,Yarra Ranges,Relationship,13cabs,Simon Purssey <simon.purssey@13cabs.com.au> (H...,NaN,"No funding, being put on backburner for this f...",NaN,No funding,NaN,NaN,...,22,2.0,1.0,19,NaN,NaN,NaN,NaN,NaN,NaN
78,Yarriambiack,NaN,Yarriambiack Shire Council,Elysia Preston <epreston@yarriambiack.vic.gov....,NaN,Have fundings. Send the cost to Elysia by emai...,Case-by-case,NaN,NaN,NaN,...,4,1.0,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
for school_name in Melbourne_schools:
    file_name = f'{school_name}_F2D_workshop_email.docx'
    template_info = extract_email_info(df_schools, school_name)
    email = create_email(template_info, my_contact_info, my_email, file_name)

C:\Users\morga\anaconda3\envs\dev\lib\site-packages\docx\styles\styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)
C:\Users\morga\anaconda3\envs\dev\lib\site-packages\docx\styles\styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)
C:\Users\morga\anaconda3\envs\dev\lib\site-packages\docx\styles\styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)
C:\Users\morga\anaconda3\envs\dev\lib\site-packages\docx\styles\styles.py:130: UserWarning: style lookup by style_id is deprecated. Use style name as key instead.
  return self._get_style_id_from_style(self[style_name], style_type)
C:\Users\morga\anaconda3\envs\dev\lib\site-packages\docx\styles\styles.p

In [41]:
def filter_schools_by_lga(df_schools, lga_list):
    """
    Filter the DataFrame for schools that belong to the specified list of LGAs.
    
    Args:
        df_schools (pd.DataFrame): DataFrame containing school data.
        lga_list (list): List of LGA names to filter schools.
    
    Returns:
        pd.DataFrame: Filtered DataFrame containing schools from the specified LGAs.
    """
    # Strip any leading/trailing whitespace from the column names
    df_schools.columns = df_schools.columns.str.strip()

    # Normalize LGA column and the LGA list to lowercase and strip whitespace
    df_schools['LGA'] = df_schools['LGA'].str.strip().str.lower()
    lga_list_normalized = [lga.strip().lower() for lga in lga_list]

    # Filter schools that belong to one of the specified LGAs
    df_filtered = df_schools[df_schools['LGA'].isin(lga_list_normalized)]
    
    return df_filtered

def filter_schools_with_at_least_1_workshop_in_4_years(df_schools):
    """
    Filter schools that have had at least 1 workshop in the last 4 years.
    
    Args:
        df_schools (pd.DataFrame): DataFrame containing school data.
    
    Returns:
        list: List of schools that meet the criteria.
    """
    # Strip any leading/trailing whitespace from the column names
    df_schools.columns = df_schools.columns.str.strip()

    # Get the current year
    current_year = datetime.now().year

    # Calculate the starting year for the 4-year period
    start_year = current_year - 4

    # Create an empty list to store schools that meet the criteria
    schools_with_at_least_1_workshop = []

    # Filter for columns that represent workshop dates (like '2020 F2D', '2021 F2D')
    # Ensure only string columns are checked
    workshop_columns = [col for col in df_schools.columns if isinstance(col, str) and ('F2D' in col or 'SR' in col)]

    # Iterate through each school in the DataFrame
    for idx, row in df_schools.iterrows():
        # Create a list to store valid workshop years
        valid_workshops = []

        # Iterate over workshop columns to check if any workshops were held in the last 4 years
        for col in workshop_columns:
            workshop_year = pd.to_datetime(row[col], errors='coerce').year if pd.notnull(row[col]) else None
            
            # If the workshop year is valid and within the last 4 years, add it to valid_workshops
            if workshop_year and workshop_year >= start_year:
                valid_workshops.append(workshop_year)

        # If the school has had at least 1 valid workshop in the last 4 years, add it to the list
        if len(valid_workshops) > 0:
            schools_with_at_least_1_workshop.append(row['Organisation Name'])

    return schools_with_at_least_1_workshop

In [53]:
lga_list = [
    'Boroondara', 
    'Manningham', 
    'Monash', 
    'Whitehorse'
]

lga = ['Hume']

filtered_df = filter_schools_by_lga(df_schools, lga_list)

priority_schools = filter_schools_with_at_least_1_workshop_in_4_years(filtered_df)

In [54]:
priority_schools

['Auburn High School',
 'Camberwell Girls Grammar School',
 'Camberwell Grammar School',
 'Rossbourne School',
 "Ruyton Girls' School",
 'Doncaster Secondary College',
 'Warrandyte High School',
 'Avila College',
 'Berengarra School Pathways Campus',
 'Mount Waverley Secondary College',
 'Box Hill High School',
 'Emmaus College',
 'Forest Hill College',
 'Koonung Secondary College',
 'Mullauna College',
 'Nunawading Christian College Secondary School',
 'Our Lady of Sion College']

In [55]:
number_of_workshops = {}
for school in priority_schools:
    template_info = extract_email_info(df_schools, school)
    number_of_workshops[template_info['School Name']] = template_info['Number of Workshops']

In [56]:
number_of_workshops

{'Auburn High School': 3,
 'Camberwell Girls Grammar School': 9,
 'Camberwell Grammar School': 3,
 'Rossbourne School': 5,
 "Ruyton Girls' School": 5,
 'Doncaster Secondary College': 7,
 'Warrandyte High School': 3,
 'Avila College': 1,
 'Berengarra School Pathways Campus': 1,
 'Mount Waverley Secondary College': 1,
 'Box Hill High School': 1,
 'Emmaus College': 4,
 'Forest Hill College': 7,
 'Koonung Secondary College': 3,
 'Mullauna College': 8,
 'Nunawading Christian College Secondary School': 1,
 'Our Lady of Sion College': 1}